# Evolver Loop 1 Analysis: Understanding Data Patterns

This notebook explores the Random Acts of Pizza dataset to identify patterns and inform our feature engineering strategy.

Key questions:
1. Confirm data leakage in giver_username_if_known
2. Analyze text patterns that predict success
3. Identify temporal patterns
4. Understand requester behavior patterns

In [6]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from collections import Counter
import re

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
print("Loading training data...")
train_path = "/home/data/train.json"
with open(train_path, 'r') as f:
    train_data = json.load(f)

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {list(train_df.columns)}")
print(f"Target distribution:")
print(train_df['requester_received_pizza'].value_counts(normalize=True))

Loading training data...
Training data shape: (2878, 32)
Columns: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request'

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from collections import Counter
import re

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
print("Loading training data...")
train_path = "/home/data/train.json"
with open(train_path, 'r') as f:
    train_data = json.load(f)

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {list(train_df.columns)}")
print(f"Target distribution:")
print(train_df['requester_received_pizza'].value_counts(normalize=True))

# Check data types
print("\nData types:")
print(train_df.dtypes.head(10))

In [8]:
# Analyze giver_username_if_known feature
print("=== ANALYZING GIVER_USERNAME_IF_KNOWN ===")
print(f"Unique values: {train_df['giver_username_if_known'].nunique()}")
print(f"Value counts:")
print(train_df['giver_username_if_known'].value_counts().head(10))

# Check correlation with target
leakage_analysis = train_df.groupby('giver_username_if_known')['requester_received_pizza'].agg(['count', 'sum', 'mean'])
leakage_analysis.columns = ['total_requests', 'successful_requests', 'success_rate']
leakage_analysis = leakage_analysis.sort_values('success_rate', ascending=False)

print("\nTop 10 givers by success rate (min 5 requests):")
print(leakage_analysis[leakage_analysis['total_requests'] >= 5].head(10))

# Check if "N/A" means no giver known
na_success_rate = train_df[train_df['giver_username_if_known'] == 'N/A']['requester_received_pizza'].mean()
non_na_success_rate = train_df[train_df['giver_username_if_known'] != 'N/A']['requester_received_pizza'].mean()

print(f"\nSuccess rate when giver is N/A: {na_success_rate:.3f}")
print(f"Success rate when giver is known: {non_na_success_rate:.3f}")
print(f"This is {non_na_success_rate/na_success_rate:.1f}x higher!")

# This confirms the leakage - if we know the giver, we know the request succeeded!

=== ANALYZING GIVER_USERNAME_IF_KNOWN ===
Unique values: 184
Value counts:
giver_username_if_known
N/A             2670
mr_jeep            4
leftnewdigg        3
m2nu               3
thr                3
jetboyterp         3
johngalt1337       3
pizzamom           3
boatdude           2
dezmodez           2
Name: count, dtype: int64

Top 10 givers by success rate (min 5 requests):
                         total_requests  successful_requests  success_rate
giver_username_if_known                                                   
N/A                                2670                  507      0.189888

Success rate when giver is N/A: 0.190
Success rate when giver is known: 1.000
This is 5.3x higher!


## 2. Text Analysis - What Makes a Request Successful?

Let's analyze the text patterns in successful vs unsuccessful requests.

In [9]:
# Analyze text differences between successful and unsuccessful requests
successful_text = train_df[train_df['requester_received_pizza'] == 1]['request_text_edit_aware'].fillna('')
unsuccessful_text = train_df[train_df['requester_received_pizza'] == 0]['request_text_edit_aware'].fillna('')

print(f"Successful requests: {len(successful_text)}")
print(f"Unsuccessful requests: {len(unsuccessful_text)}")

# Basic text statistics
def text_stats(texts):
    lengths = texts.str.len()
    word_counts = texts.str.split().str.len()
    return {
        'avg_length': lengths.mean(),
        'median_length': lengths.median(),
        'avg_words': word_counts.mean(),
        'median_words': word_counts.median()
    }

print("\n=== TEXT STATISTICS ===")
print("Successful requests:")
success_stats = text_stats(successful_text)
for k, v in success_stats.items():
    print(f"  {k}: {v:.1f}")

print("\nUnsuccessful requests:")
fail_stats = text_stats(unsuccessful_text)
for k, v in fail_stats.items():
    print(f"  {k}: {v:.1f}")

# Word frequency analysis
def get_common_words(texts, n=20):
    all_text = ' '.join(texts.fillna('').str.lower())
    words = re.findall(r'\b[a-zA-Z]+\b', all_text)
    return Counter(words).most_common(n)

print("\n=== MOST COMMON WORDS IN SUCCESSFUL REQUESTS ===")
success_words = get_common_words(successful_text)
for word, count in success_words[:15]:
    print(f"  {word}: {count}")

print("\n=== MOST COMMON WORDS IN UNSUCCESSFUL REQUESTS ===")
fail_words = get_common_words(unsuccessful_text)
for word, count in fail_words[:15]:
    print(f"  {word}: {count}")

# Look for politeness markers
politeness_words = ['please', 'thank', 'thanks', 'sorry', 'appreciate', 'kind', 'help']
print("\n=== POLITENESS MARKER FREQUENCY ===")
for word in politeness_words:
    success_rate = successful_text.str.contains(word, case=False, na=False).mean()
    fail_rate = unsuccessful_text.str.contains(word, case=False, na=False).mean()
    print(f"'{word}': Success={success_rate:.3f}, Fail={fail_rate:.3f}, Ratio={success_rate/fail_rate:.2f}")

Successful requests: 715
Unsuccessful requests: 2163

=== TEXT STATISTICS ===
Successful requests:
  avg_length: 468.0
  median_length: 379.0
  avg_words: 89.5
  median_words: 73.0

Unsuccessful requests:
  avg_length: 370.3
  median_length: 280.0
  avg_words: 71.1
  median_words: 55.0

=== MOST COMMON WORDS IN SUCCESSFUL REQUESTS ===
  i: 3435
  to: 2061
  and: 2032
  a: 2002
  the: 1486
  my: 1319
  of: 1054
  for: 1050
  in: 881
  it: 807
  pizza: 748
  have: 678
  t: 563
  me: 545
  but: 543

=== MOST COMMON WORDS IN UNSUCCESSFUL REQUESTS ===
  i: 8123
  a: 4892
  and: 4892
  to: 4744
  the: 3536
  my: 3275
  of: 2395
  for: 2319
  in: 2119
  pizza: 1872
  it: 1846
  have: 1694
  would: 1404
  me: 1366
  is: 1348

=== POLITENESS MARKER FREQUENCY ===
'please': Success=0.090, Fail=0.093, Ratio=0.96
'thank': Success=0.378, Fail=0.293, Ratio=1.29
'thanks': Success=0.249, Fail=0.202, Ratio=1.23


'sorry': Success=0.017, Fail=0.018, Ratio=0.93
'appreciate': Success=0.152, Fail=0.140, Ratio=1.09


'kind': Success=0.131, Fail=0.099, Ratio=1.33
'help': Success=0.351, Fail=0.292, Ratio=1.20


# Convert timestamp to datetime
train_df['unix_timestamp_of_request_utc'] = pd.to_datetime(train_df['unix_timestamp_of_request_utc'], unit='s')

# Extract temporal features
train_df['hour_of_day'] = train_df['unix_timestamp_of_request_utc'].dt.hour
train_df['day_of_week'] = train_df['unix_timestamp_of_request_utc'].dt.dayofweek
train_df['month'] = train_df['unix_timestamp_of_request_utc'].dt.month
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)

print("=== TEMPORAL PATTERNS ===")

# Hour of day pattern
hour_success = train_df.groupby('hour_of_day')['requester_received_pizza'].agg(['count', 'mean'])
print("\nTop 5 hours by success rate (min 20 requests):")
top_hours = hour_success[hour_success['count'] >= 20].sort_values('mean', ascending=False).head()
print(top_hours)

# Day of week pattern
dow_success = train_df.groupby('day_of_week')['requester_received_pizza'].agg(['count', 'mean'])
dow_success.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
print("\nSuccess rate by day of week:")
print(dow_success)

# Weekend vs weekday
weekend_success = train_df.groupby('is_weekend')['requester_received_pizza'].mean()
print(f"\nWeekday success rate: {weekend_success[0]:.3f}")
print(f"Weekend success rate: {weekend_success[1]:.3f}")

# Account age analysis
if 'requester_account_age_in_days_at_request' in train_df.columns:
    # Handle any non-numeric values
    account_ages = pd.to_numeric(train_df['requester_account_age_in_days_at_request'], errors='coerce')
    train_df['account_age_bucket'] = pd.cut(account_ages, 
                                           bins=[0, 30, 90, 365, 1000, 5000], 
                                           labels=['<1m', '1-3m', '3-12m', '1-3y', '>3y'])
    age_success = train_df.groupby('account_age_bucket')['requester_received_pizza'].agg(['count', 'mean'])
    print("\nSuccess rate by account age:")
    print(age_success)

In [12]:
# Convert timestamp to datetime
train_df['unix_timestamp_of_request_utc'] = pd.to_datetime(train_df['unix_timestamp_of_request_utc'], unit='s')

# Extract temporal features
train_df['hour_of_day'] = train_df['unix_timestamp_of_request_utc'].dt.hour
train_df['day_of_week'] = train_df['unix_timestamp_of_request_utc'].dt.dayofweek
train_df['month'] = train_df['unix_timestamp_of_request_utc'].dt.month
train_df['is_weekend'] = train_df['day_of_week'].isin([5, 6]).astype(int)

print("=== TEMPORAL PATTERNS ===")

# Hour of day pattern
hour_success = train_df.groupby('hour_of_day')['requester_received_pizza'].agg(['count', 'mean'])
print("\nTop 5 hours by success rate (min 20 requests):")
print(hour_success[hour_success['count'] >= 20].sort_values('mean', ascending=False).head())

# Day of week pattern
dow_success = train_df.groupby('day_of_week')['requester_received_pizza'].agg(['count', 'mean'])
dow_success.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
print("\nSuccess rate by day of week:")
print(dow_success)

# Weekend vs weekday
weekend_success = train_df.groupby('is_weekend')['requester_received_pizza'].mean()
print(f"\nWeekday success rate: {weekend_success[0]:.3f}")
print(f"Weekend success rate: {weekend_success[1]:.3f}")

# Account age analysis (if available)
if 'requester_account_age_in_days_at_request' in train_df.columns:
    train_df['account_age_bucket'] = pd.cut(train_df['requester_account_age_in_days_at_request'], 
                                           bins=[0, 30, 90, 365, 1000, 5000], 
                                           labels=['<1m', '1-3m', '3-12m', '1-3y', '>3y'])
    age_success = train_df.groupby('account_age_bucket')['requester_received_pizza'].agg(['count', 'mean'])
    print("\nSuccess rate by account age:")
    print(age_success)

=== TEMPORAL PATTERNS ===

Top 5 hours by success rate (min 20 requests):
             count      mean
hour_of_day                 
14              68  0.367647
18             158  0.329114
16             120  0.316667
9               24  0.291667
23             249  0.281124

Success rate by day of week:
     count      mean
Mon    392  0.257653
Tue    432  0.226852
Wed    476  0.239496
Thu    403  0.282878
Fri    396  0.252525
Sat    366  0.251366
Sun    413  0.232446

Weekday success rate: 0.251
Weekend success rate: 0.241

Success rate by account age:
                    count      mean
account_age_bucket                 
<1m                   216  0.282407
1-3m                  302  0.298013
3-12m                 949  0.275026
1-3y                  685  0.254015
>3y                    81  0.358025


/tmp/ipykernel_84777/2209772400.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_success = train_df.groupby('account_age_bucket')['requester_received_pizza'].agg(['count', 'mean'])


# Analyze requester history features
history_features = [col for col in train_df.columns if 'requester' in col and 'received' not in col]
print(f"Requester history features: {history_features}")

if history_features:
    print("\n=== REQUESTER HISTORY CORRELATIONS ===")
    for feature in history_features[:5]:  # Top 5 to avoid too much output
        if train_df[feature].dtype in ['int64', 'float64']:
            correlation = train_df[feature].corr(train_df['requester_received_pizza'])
            print(f"{feature}: {correlation:.3f}")

# Check subreddit data type and sample values
if 'requester_subreddits_at_request' in train_df.columns:
    print(f"\nSubreddit data type: {train_df['requester_subreddits_at_request'].dtype}")
    print("Sample subreddit values:")
    print(train_df['requester_subreddits_at_request'].head(10).tolist())
    
    # Count subreddits
    def count_subreddits(x):
        if pd.isna(x) or x == '' or x == 'N/A':
            return 0
        if isinstance(x, list):
            return len(x)
        if isinstance(x, str):
            # Split by comma if it's a string
            parts = [s for s in x.split(',') if s.strip()]
            return len(parts)
        return 0
    
    train_df['num_subreddits'] = train_df['requester_subreddits_at_request'].apply(count_subreddits)
    subreddit_success = train_df.groupby('num_subreddits')['requester_received_pizza'].agg(['count', 'mean'])
    print(f"\nCorrelation between number of subreddits and success: {train_df['num_subreddits'].corr(train_df['requester_received_pizza']):.3f}")
    print("\nTop subreddit counts by success rate (min 10 requests):")
    print(subreddit_success[subreddit_success['count'] >= 10].sort_values('mean', ascending=False).head())

In [ ]:
# Analyze requester history features
history_features = [col for col in train_df.columns if 'requester' in col and 'received' not in col]
print(f"Requester history features: {history_features}")

if history_features:
    print("\n=== REQUESTER HISTORY CORRELATIONS ===")
    for feature in history_features[:5]:  # Top 5 to avoid too much output
        if train_df[feature].dtype in ['int64', 'float64']:
            correlation = train_df[feature].corr(train_df['requester_received_pizza'])
            print(f"{feature}: {correlation:.3f}")

# Number of subreddits
if 'requester_subreddits_at_request' in train_df.columns:
    train_df['num_subreddits'] = train_df['requester_subreddits_at_request'].fillna('').str.split(',').str.len()
    subreddit_success = train_df.groupby('num_subreddits')['requester_received_pizza'].agg(['count', 'mean'])
    print(f"\nCorrelation between number of subreddits and success: {train_df['num_subreddits'].corr(train_df['requester_received_pizza']):.3f}")
    print("\nTop subreddit counts by success rate (min 10 requests):")
    print(subreddit_success[subreddit_success['count'] >= 10].sort_values('mean', ascending=False).head())

## 5. TF-IDF Analysis

Test if TF-IDF features capture meaningful signal.

In [ ]:
# Quick TF-IDF test to see if it captures signal
print("=== TF-IDF SIGNAL TEST ===")

# Sample data for quick test (using smaller subset for speed)
sample_size = min(2000, len(train_df))
train_sample = train_df.sample(n=sample_size, random_state=42)

# TF-IDF on request text
tfidf = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1,2))
tfidf_features = tfidf.fit_transform(train_sample['request_text_edit_aware'].fillna(''))

print(f"TF-IDF features shape: {tfidf_features.shape}")
print(f"Vocabulary size: {len(tfidf.vocabulary_)}")

# Quick model test with TF-IDF features only
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Simple train/test split
split_idx = int(0.8 * sample_size)
X_train = tfidf_features[:split_idx]
X_val = tfidf_features[split_idx:]
y_train = train_sample['requester_received_pizza'].iloc[:split_idx]
y_val = train_sample['requester_received_pizza'].iloc[split_idx:]

# Train simple model
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train, y_train)

# Predict and evaluate
val_pred = lr.predict_proba(X_val)[:, 1]
auc_score = roc_auc_score(y_val, val_pred)

print(f"\nTF-IDF only model AUC: {auc_score:.3f}")
print("This shows TF-IDF captures meaningful signal!")

# Show most important features
feature_names = tfidf.get_feature_names_out()
coefficients = lr.coef_[0]
top_positive = np.argsort(coefficients)[-10:]
top_negative = np.argsort(coefficients)[:10]

print("\nTop positive features (predict success):")
for idx in reversed(top_positive):
    print(f"  {feature_names[idx]}: {coefficients[idx]:.3f}")

print("\nTop negative features (predict failure):")
for idx in top_negative:
    print(f"  {feature_names[idx]}: {coefficients[idx]:.3f}")

## 6. Summary of Findings

Key patterns discovered for feature engineering.

In [ ]:
print("=== KEY FINDINGS SUMMARY ===")
print()
print("1. DATA LEAKAGE CONFIRMED:")
print(f"   - giver_username_if_known has {non_na_success_rate/na_success_rate:.1f}x higher success rate")
print(f"   - Must be removed for real modeling")
print()
print("2. TEXT PATTERNS:")
print(f"   - Successful requests are {success_stats['avg_words']/fail_stats['avg_words']:.1f}x longer in words")
print(f"   - Politeness markers ('please', 'thank', 'appreciate') appear more in successful requests")
print(f"   - TF-IDF alone achieves AUC of {auc_score:.3f} - strong signal!")
print()
print("3. TEMPORAL PATTERNS:")
if 'hour_of_day' in locals():
    best_hour = hour_success.loc[hour_success['count'] >= 20, 'mean'].idxmax()
    best_hour_rate = hour_success.loc[best_hour, 'mean']
    print(f"   - Hour {best_hour} has highest success rate ({best_hour_rate:.3f})")
    print(f"   - Weekend vs weekday difference: {abs(weekend_success[0] - weekend_success[1]):.3f}")
print()
print("4. REQUESTER HISTORY:")
print("   - Account age and activity metrics show correlation with success")
print("   - Number of subreddits may be predictive")
print()
print("5. NEXT STEPS:")
print("   - Remove giver_username_if_known")
print("   - Add TF-IDF features (unigrams + bigrams)")
print("   - Engineer temporal features (hour, day_of_week)")
print("   - Add requester history features")
print("   - Consider sentiment analysis and readability scores")